<a href="https://colab.research.google.com/github/wahyufaith/Computer-Vision/blob/main/Save_and_restore_model_(h3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**WAHYU PEBRIANTO**

In [1]:
!pip install pyyaml h5py

In [13]:
import os
import tensorflow as tf
from tensorflow import keras
print(tf.version.VERSION)

2.5.0


In [17]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

#print(len(train_images))
#print(len(test_images))

In [19]:
train_labels = train_labels[:1000]
test_labels  = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28)/255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

**Mendefinisak model**

In [29]:
def create_model():
  model = tf.keras.Sequential([
                               keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                               keras.layers.Dropout(0.2),
                               keras.layers.Dense(10)
                               ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])
  return model

model = create_model()

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# **1. MENYIMPAN WEIGHT MODEL YANG DI TRAINING**

# **Save checkpoints during training**

1. Kita dapat menggunakan per-trained model tanpa harus mentraining kembali, atau mengambil training yang Kita tinggalkan jika proses training terganggu. **(The tf.keras.callbacks.ModelCheckpointcallback)** memungkinkan kita untuk terus menyimpan model baik selama proses dan pada akhir training.

**Checkpoint callback usage**

Create a (tf.keras.callbacks.ModelCheckpoint) callback that saves weights only during training:

In [30]:
checkpoint_path = "/content/sample_data/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - 1s 9ms/step - loss: 1.1279 - sparse_categorical_accuracy: 0.6930 - val_loss: 0.7135 - val_sparse_categorical_accuracy: 0.7720

Epoch 00001: saving model to /content/sample_data/training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.4095 - sparse_categorical_accuracy: 0.8940 - val_loss: 0.5732 - val_sparse_categorical_accuracy: 0.8130

Epoch 00002: saving model to /content/sample_data/training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 6ms/step - loss: 0.3061 - sparse_categorical_accuracy: 0.9180 - val_loss: 0.4575 - val_sparse_categorical_accuracy: 0.8530

Epoch 00003: saving model to /content/sample_data/training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 5ms/step - loss: 0.2052 - sparse_categorical_accuracy: 0.9530 - val_loss: 0.4384 - val_sparse_categorical_accuracy: 0.8560

Epoch 00004: saving model to /content/sample_data/training_1/cp.ckpt
Epoch 5/

Tindakan ini akan membuat satu kumpulan TensorFlow checkpoint files yang diperbarui di akhir setiap epoch:

In [31]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

Selama dua model berbagi arsitektur yang sama, Anda dapat berbagi weight di antara keduanya. Jadi, saat memulihkan model dari weight, buat model dengan arsitektur yang sama dengan model aslinya, lalu atur weightnya.

Sekarang buat kembali model baru yang tidak terlatih dan evaluasi pada set pengujian. Model yang tidak terlatih akan tampil pada tingkat kebetulan (~10% accuracy):

In [32]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3175 - sparse_categorical_accuracy: 0.1280
Untrained model, accuracy: 12.80%


**Kemudian muat weight dari checkpoint dan evaluasi kembali:**

In [33]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4234 - sparse_categorical_accuracy: 0.8640
Restored model, accuracy: 86.40%


# **2. MENYIMPAN MODEL YANG DI TRAINING PADA SETIAP EPOCH/SIMPAN EPOCH MODEL**


Latih model baru, dan simpan checkpoint bernama unik setiap 5 epoch:

In [34]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "/content/sample_data/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


Sekarang, lihat checkpoint yang dihasilkan dan pilih yang terbaru:

In [35]:
os.listdir(checkpoint_dir)

['cp-0035.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0045.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'checkpoint',
 'cp-0040.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001']

In [36]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

**Catatan: format TensorFlow default hanya menyimpan 5 checkpoint terbaru.**

Untuk menguji, setel ulang model dan muat checkpoint terbaru:

In [37]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4865 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


##**Apa saja file-file ini?**
Kode di atas menyimpan weight ke kumpulan file berformat checkpoint yang hanya berisi weight terlatih dalam format biner. checkpoint berisi:

- Satu atau beberapa pecahan yang berisi weight model Kita.
- File indeks yang menunjukkan weight mana yang disimpan.


Jika Kita melatih model pada satu mesin, kita akan memiliki satu pecahan dengan akhiran: **.data-00000-of-00001**

##**Simpan bobot secara manual Manual**

Menyimpan weight secara manual dengan **Model.save_weightsmetode.** Secara default, **tf.keras—** dan **save_weights** khususnya—menggunakan format checkpoint TensorFlow dengan **.ckpt** ekstensi (menyimpan dalam **HDF5** dengan **.h5** ekstensi tercakup dalam panduan Menyimpan dan membuat serial model ):

In [38]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4865 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


# **3. Simpan seluruh model**

format jika kita akan menyimpan seluruh file model yaitu : **(SavedModel dan HDF5)**.

###**a).** Format **SavedModel**


Format **SavedModel** adalah cara lain untuk membuat serial model. Model yang disimpan dalam format ini dapat dipulihkan menggunakan **tf.keras.models.load_model** dan kompatibel dengan **TensorFlow Serving**.

**SavedModel** panduan masuk ke detail tentang bagaimana serving/memeriksa **SavedModel**. Bagian di bawah ini mengilustrasikan langkah-langkah untuk save dan merestore model.

In [39]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
#!mkdir -p saved_model
model.save('/content/sample_data/saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1562 - sparse_categorical_accuracy: 0.6510
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4299 - sparse_categorical_accuracy: 0.8770
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2829 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2022 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1429 - sparse_categorical_accuracy: 0.9740
INFO:tensorflow:Assets written to: /content/sample_data/saved_model/my_model/assets


Format **SavedModel** adalah direktori yang berisi **biner protobuf** dan **checpoint TensorFlow**.

Periksa direktori model yang disimpan:

In [46]:
# my_model directory
#!ls /content/sample_data/saved_model/

# Contains an assets folder, saved_model.pb, and variables folder.
!ls /content/sample_data/saved_model/my_model

assets	keras_metadata.pb  saved_model.pb  variables


**Setelah kita menyimpan model, coba muat ulang model Keras baru dari model yang telah disimpan:**

In [47]:
new_model = tf.keras.models.load_model('/content/sample_data/saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


**Model yang dipulihkan dikompilasi dengan argumen yang sama dengan model aslinya. Coba jalankan evaluasi dan prediksi dengan model yang dimuat:**

In [48]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4152 - sparse_categorical_accuracy: 0.8640
Restored model, accuracy: 86.40%
(1000, 10)


##**b).** Format **HDF5**


Keras menyediakan format penyimpanan dasar menggunakan standar HDF5 .

In [49]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('/content/sample_data/my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1970 - sparse_categorical_accuracy: 0.6630
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4401 - sparse_categorical_accuracy: 0.8620
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3054 - sparse_categorical_accuracy: 0.9140
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2253 - sparse_categorical_accuracy: 0.9460
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1463 - sparse_categorical_accuracy: 0.9680


**Sekarang, buat ulang model dari file itu:**

In [51]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('/content/sample_data/my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


**Periksa keakuratannya:**

In [52]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4253 - sparse_categorical_accuracy: 0.8650
Restored model, accuracy: 86.50%


##KETERANGAN
Perbedaan utama antara **HDF5** dan **SavedModel** adalah bahwa **HDF5** menggunakan konfigurasi objek untuk menyimpan arsitektur model, sedangkan **SavedModel** menyimpan grafik eksekusi. Dengan demikian, **SavedModels** dapat menyimpan objek khusus seperti model subkelas dan lapisan khusus tanpa memerlukan kode asli.